# 前処理

In [1]:
from captum.attr import *
import torch

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import BertTokenizer

from torch.utils.data import TensorDataset

# Load the IMDB dataset
import datasets

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the IMDB dataset
train_dataset, test_dataset = datasets.load_dataset('imdb', split=['train', 'test'])

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 450.78it/s]


In [3]:
train_dataset = train_dataset[:1000]
test_dataset = test_dataset[:1000]

In [4]:
# Tokenize the texts
train_encodings = tokenizer(train_dataset['text'], truncation=True, padding=True)
test_encodings = tokenizer(test_dataset['text'], truncation=True, padding=True)

In [5]:
# Convert labels to tensors
train_labels = torch.tensor(train_dataset['label'])
test_labels = torch.tensor(test_dataset['label'])

# Create TensorDatasets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    train_labels
)

test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    test_labels
)

In [6]:
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader

# BERTモデルを読み込む
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [7]:
from tqdm import tqdm

# データローダーを作成する
batch_size = 32  # 適宜調整する
train_dataset
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# オプティマイザーを設定する
optimizer = AdamW(model.parameters(), lr=2e-5)

# モデルを訓練する
num_epochs = 10  # 適宜調整する

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        input_ids = batch[0].to(model.device).cuda()
        attention_masks = batch[1].to(model.device).cuda()
        labels = batch[2].to(model.device).cuda()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(avg_loss)

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:15<00:00,  2.11it/s]


0.1535117942839861


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.26it/s]


0.01300091047596652


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.26it/s]


...

0.001109989427277469


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.25it/s]


0.0008749525168241234


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.25it/s]

0.0006993858769419603


# LIME の基本実装

In [8]:
import numpy as np
from sklearn.linear_model import Ridge

class Lime:
    def __init__(self, predict_fn, perturb_fn, kernel_width=0.75):
        self.predict_fn = predict_fn
        self.perturb_fn = perturb_fn
        self.kernel_width = kernel_width

    def explain_instance(self, instance, use_weights=True):
        # Step 1: Generate random samples around the instance
        samples = self.perturb_fn(instance)

        # Step 2: Get predictions for the samples using the black-box model
        predictions = [self.predict_fn(sample) for sample in tqdm(samples)]

        if use_weights:
            # Step 3: Compute distances between the instance and the samples
            distances = self._compute_distances(instance, samples)
            # Step 4: Compute weights using kernel function
            weights = self._kernel(distances)
        else:
            weights = np.ones(samples.shape[0])

        # Step 5: Fit a linear model using the samples and predictions
        model = Ridge(alpha=1.0)
        model.fit(samples, predictions, sample_weight=weights)

        return model.coef_

    def _compute_distances(self, instance, samples):
        return np.linalg.norm(samples - instance, axis=1)

    def _kernel(self, distances):
        return np.exp(-distances / self.kernel_width)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [9]:
def additive_fn(instance, num_samples=1000):
    samples = np.random.normal(0, 1, (num_samples, instance.shape[0]))
    return samples + instance

In [10]:
def perturb_fn(instance, num_samples=1000, perturb_size=1):
    mask_array = []
    for _ in range(num_samples):
        row = [1.0 if idx >= perturb_size else 0.0 for idx in range(instance.shape[0])]
        mask_array.append(np.random.permutation(row))
    mask_array = np.array(mask_array)
    return mask_array * instance

In [11]:
comp_fn = lambda x: perturb_fn(x, perturb_size=1)
suff_fn = lambda x: perturb_fn(x, perturb_size=4)

In [12]:
# Define a black-box model
def black_box_model(array):
    predictions = array[0] + array[1] * 2 + array[2] * 3
    predictions = array[3] ** 2 + array[4] ** 3
    return predictions

# ベースライン実験
動作確認兼ベースライン生成のコード

In [13]:
# Explain an instance
instance = np.array([1, 1, 1, 1, 1])
# Create an instance of LIME
lime = Lime(predict_fn=black_box_model, perturb_fn=additive_fn)
explanation = lime.explain_instance(instance, use_weights=True)
print(explanation)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 298569.48it/s]

[0.10577409 0.06916762 0.12131477 1.94136409 4.68235533]


# Faithfulness実験
ここで解釈モデルが Faithfulness を予測するモデルになっているはず...

In [14]:
# Explain an instance
instance = np.array([1, 1, 1, 1, 1])
# Create an instance of LIME
lime = Lime(predict_fn=black_box_model, perturb_fn=comp_fn)
explanation = lime.explain_instance(instance, use_weights=True)
print(explanation)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 502853.85it/s]

[-0.39294367 -0.39216856 -0.39239663  0.58889633  0.58861254]


# BERT でやってみる

In [20]:
def to_embedding(sample):
    ids, mask = sample[0].unsqueeze(0).cuda(), sample[1].unsqueeze(0).cuda()
    embedding = model.bert.embeddings(input_ids=ids)
    return embedding.cpu().squeeze(0).detach().numpy().flatten()

In [21]:
def reshape(array):
    array = array.reshape(-1, 768)
    return array

In [22]:
def bert_wrapper(instance):
    reshaped = reshape(instance)
    with torch.no_grad():
        reshaped = torch.tensor(reshaped).unsqueeze(0).cuda().to(torch.float32)
        prediction = model(inputs_embeds=reshaped)
    return prediction

In [23]:
instance = to_embedding(train_dataset[0])
lime = Lime(predict_fn=bert_wrapper, perturb_fn=additive_fn)
explanation = lime.explain_instance(instance, use_weights=True)
print(explanation)


 37%|████████████████████████████████████████████████████████████████████████▋                                                                                                                             | 367/1000 [00:03<00:06, 96.82it/s]


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                 | 747/1000 [00:07<00:02, 101.09it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 102.30it/s]


TypeError: float() argument must be a string or a number, not 'SequenceClassifierOutput'